<a href="https://colab.research.google.com/github/apkolla/clustering_usequity_fundamentals/blob/main/MissingData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import pandas as pd
import requests
import time
import bs4 as bs
import datetime as dt
import io
import numpy as np
import time

In [ ]:
df = pd.read_csv('fundamentals.csv')
df.drop(df.iloc[:, 0:1], inplace=True, axis=1)

In [ ]:
print(df.shape)
print(df.dtypes)
print(df.isnull().sum())

(500, 31)
Symbol                         object
MarketCapitalization            int64
EBITDA                         object
PERatio                        object
PEGRatio                       object
BookValue                      object
DividendPerShare               object
DividendYield                  object
EPS                           float64
GrossProfitTTM                  int64
DilutedEPSTTM                 float64
QuarterlyEarningsGrowthYOY    float64
QuarterlyRevenueGrowthYOY     float64
TrailingPE                     object
Beta                           object
fiscalDateEnding               object
grossProfit                    object
totalRevenue                   object
costOfRevenue                  object
operatingIncome                 int64
operatingExpenses              object
ebit                           object
netIncome                       int64
totalAssets                     int64
totalLiabilities                int64
currentDebt                    object
to

Every feature should have imported as an integer or a float at the very least. Given there are no missing values, I am guessing the features imported with some other string. So lets take a look.


In [ ]:
df.head()

,Symbol,MarketCapitalization,EBITDA,PERatio,PEGRatio,BookValue,DividendPerShare,DividendYield,EPS,GrossProfitTTM,...,ebit,netIncome,totalAssets,totalLiabilities,currentDebt,totalShareholderEquity,operatingCashflow,capitalExpenditures,profitLoss,dividendPayout
0,MMM,51511656000,-6610999808,None,4.153,8.7,6,0.065,-12.63,15000000000,...,-8744000000,-6995000000,50580000000,45712000000,2947000000,4807000000,6680000000,1615000000,-6979000000,3311000000
1,AOS,11934819000,835700000,21.83,1.984,12.34,1.22,0.0159,3.69,1329600000,...,233100000,235700000,3332300000,1584600000,10000000,1747700000,391400000,70300000,235700000,177200000
2,ABT,195185590000,9933000000,34.3,26.92,21.59,2.08,0.0197,3.26,24579000000,...,8864000000,6933000000,74438000000,37752000000,2251000000,36686000000,9581000000,1777000000,15239000000,3309000000
3,ABBV,308598571000,26254332000,64,1.622,6.85,5.99,0.0356,2.72,41527000000,...,15698000000,11836000000,138805000000,121551000000,4136000000,17254000000,24943000000,695000000,11845000000,10043000000
4,ACN,233034121000,11303738000,34.48,2.508,42.55,4.82,0.0139,10.78,20731607000,...,9054884000,6871557000,51245305000,25552466000,104810000,25692839000,9524268000,528172000,16142862000,2827394000


It looks like we have 'None' instead of NaN values, which is making the columns string. So, lets remove those strings and replace them with NaN. Then I will create a dataframe of the missing features and their value counts. From there I will write a function to loop through the original dataframe and collect the Symbol in a dictionary for each of the missing features.



In [ ]:
df = df.replace('None',np.nan)
na_features = df.isna().sum()
cols = ['feature','na_count']
m_feats = pd.DataFrame(na_features).reset_index()
m_feats.columns = cols
missing = m_feats[m_feats['na_count']>0]
missing.sort_values(by='na_count',ascending=False).head(20)

,feature,na_count
7,DividendYield,97
6,DividendPerShare,97
25,currentDebt,95
30,dividendPayout,81
3,PERatio,30
2,EBITDA,28
28,capitalExpenditures,12
4,PEGRatio,5
14,Beta,3
29,profitLoss,3


There are 17 features with missing values to varying degrees. Some of these will be easier to deal with than others. Lets start by looking at the dividend payout to get an idea of which companies even pay dividends

In [ ]:
missing_divs = df.loc[df['DividendPerShare'].isna(),'Symbol'].tolist()



In [ ]:
import yfinance as yf

In [ ]:
div = {}
for ticker in missing_divs:
    ticker_object = yf.Ticker(ticker)
    if  ticker_object.dividends.empty:
      continue
    else:
      ticker_div = ticker_object.dividends[::-1][0]
      div[ticker]= ticker_div



In [ ]:
dps = pd.DataFrame.from_dict(div,orient='index', columns=['DividendPerShare']).reset_index()
dps.columns = ['Symbol','DividendPerShare']
dps.head()

,Symbol,DividendPerShare
0,ADBE,0.0065
1,AAL,0.1000
2,APTV,0.2200
3,ADSK,0.0150
4,BA,2.0550


In [ ]:
df = df.merge(dps,how='left', left_on = 'Symbol', right_on = 'Symbol')
df.columns

Index(['Symbol', 'MarketCapitalization', 'EBITDA', 'PERatio', 'PEGRatio',
       'BookValue', 'DividendPerShare_x', 'DividendYield', 'EPS',
       'GrossProfitTTM', 'DilutedEPSTTM', 'QuarterlyEarningsGrowthYOY',
       'QuarterlyRevenueGrowthYOY', 'TrailingPE', 'Beta', 'fiscalDateEnding',
       'grossProfit', 'totalRevenue', 'costOfRevenue', 'operatingIncome',
       'operatingExpenses', 'ebit', 'netIncome', 'totalAssets',
       'totalLiabilities', 'currentDebt', 'totalShareholderEquity',
       'operatingCashflow', 'capitalExpenditures', 'profitLoss',
       'dividendPayout', 'DividendPerShare_y'],
      dtype='object')

In [ ]:
df['DividendPerShare'] = df['DividendPerShare_x'].fillna(df['DividendPerShare_y'])
df.DividendPerShare.isna().sum()

75

I am left with 76 companies with no dividend per share information from either of the two sources. Spot checking a few, and trusting two sources of data, I can confidetially say that these companies do not pay dividends, and I will fill their values in with a 0.

In [ ]:
df.DividendPerShare.fillna(0,inplace=True)
df = df.drop(['DividendPerShare_x','DividendPerShare_y'],axis=1)
df['DividendPerShare'] = df.DividendPerShare.astype(float)
df.DividendPerShare.describe()

count    500.000000
mean       2.209233
std        2.445051
min        0.000000
25%        0.500000
50%        1.605000
75%        3.127500
max       20.000000
Name: DividendPerShare, dtype: float64

Now that I have DPS information for all stocks, I can go through and calculate the dividend yield for the companies who have missing values. To do this I will have to pull in a historical price point for the equities who are missing data, and I will pull in the closing price on the last trading day of 2023 which was 12/29

In [ ]:
missing_dy_tickers =  df.loc[df['DividendYield'].isna(),'Symbol'].to_list()
hist = yf.download(missing_dy_tickers, start="2023-12-29", end="2023-12-30")



[*********************100%%**********************]  97 of 97 completed


In [ ]:
history = hist.stack().reset_index().rename(index=str, columns={"level_1": "Ticker"}).sort_values(['Ticker','Date'])
history.reset_index()
history = history[['Ticker','Close']]
history.head()

Price,Ticker,Close
0,AAL,13.740000
1,ABNB,136.139999
2,ACGL,74.269997
3,ADBE,596.599976
4,ADSK,243.479996


In [ ]:
history = history.merge(df[['Symbol','DividendPerShare']],how='left',left_on='Ticker',right_on='Symbol')
history['DividendYield'] = history['Close']/history['DividendPerShare']
history = history.replace(np.inf,0)
history.head()


,Ticker,Close,Symbol,DividendPerShare,DividendYield
0,AAL,13.740000,AAL,0.1000,137.399998
1,ABNB,136.139999,ABNB,0.0000,0.000000
2,ACGL,74.269997,ACGL,0.0000,0.000000
3,ADBE,596.599976,ADBE,0.0065,91784.611629
4,ADSK,243.479996,ADSK,0.0150,16231.999715


In [ ]:
df = df.merge(history[['Ticker','DividendYield']],how='left', left_on = 'Symbol', right_on = 'Ticker')
df['DividendYield'] = df['DividendYield_x'].fillna(df['DividendYield_y'])
df = df.drop(['DividendYield_x','DividendYield_y','Ticker'],axis=1)

Now, I am going to deal with the PE Ratio and PEG Ratios. Financial literature (source) indicates that that PEG Ratio is a better unit of comparison across companies in different industries and different growth rates than the PE ratio. For this reason, I will simply drop the PE Ratio from the analysis and move forward with the PEG.

In [ ]:
df = df.drop('PERatio',axis=1)

In [ ]:
#fill get data for missing PEG
missing_PEG =  df.loc[df['PEGRatio'].isna(),'Symbol'].to_list()
PEG = {}
#tickers = yf.Tickers(missing_ebitda)
#ebitda_df = pd.DataFrame(columns = ['webiste','ebitda','Symbol'])
for ticker in missing_PEG:
  t = yf.Ticker(ticker)
  x = t.info
  y = pd.DataFrame(x)
  if 'pegRatio' not in y:
    continue
  else:
    #print(y.pegRatio)
    z = y.pegRatio.mean()
    PEG[ticker] = z


PEGR = pd.DataFrame.from_dict(PEG,orient='index', columns=['PEGRatio']).reset_index()
PEGR.columns = ['Symbol','PEGRatio']
df = df.merge(PEGR[['Symbol','PEGRatio']],how='left', left_on = 'Symbol', right_on = 'Symbol')
df['PEGRatio'] = df['PEGRatio_x'].fillna(df['PEGRatio_y'])
df = df.drop(['PEGRatio_x','PEGRatio_y'],axis=1)

In [ ]:
missing_PEG =  df.loc[df['PEGRatio'].isna(),'Symbol'].to_list()
missing_PEG


['VLTO']

I am still missing the PEG for VLTO. For this I will manually look up the value from schwab.com and fill it in

In [ ]:
df['PEGRatio'] = df.PEGRatio.fillna(4.56)

Same Process for EBITDA

In [ ]:
#fill get data for missing PEG
missing_EBITDA =  df.loc[df['EBITDA'].isna(),'Symbol'].to_list()
EBITDA = {}
#tickers = yf.Tickers(missing_ebitda)
#ebitda_df = pd.DataFrame(columns = ['webiste','ebitda','Symbol'])
for ticker in missing_PEG:
  t = yf.Ticker(ticker)
  x = t.info
  y = pd.DataFrame(x)
  if 'ebitda' not in y:
    continue
  else:
    #print(y.pegRatio)
    z = y.ebitda.mean()
    EBITDA[ticker] = z


EBITDA = pd.DataFrame.from_dict(EBITDA,orient='index', columns=['EBITDA']).reset_index()
EBITDA.columns = ['Symbol','EBITDA']
df = df.merge(EBITDA[['Symbol','EBITDA']],how='left', left_on = 'Symbol', right_on = 'Symbol')
df['EBITDA'] = df['EBITDA_x'].fillna(df['EBITDA_y'])
df = df.drop(['EBITDA_x','EBITDA_y'],axis=1)

In [ ]:
missing_EBITDA =  df.loc[df['EBITDA'].isna(),'Symbol'].to_list()
missing_EBITDA

['AXP',
 'BAC',
 'BK',
 'BX',
 'COF',
 'SCHW',
 'C',
 'CFG',
 'CMA',
 'DFS',
 'EG',
 'FITB',
 'GS',
 'HBAN',
 'JPM',
 'KEY',
 'MTB',
 'MS',
 'NTRS',
 'PNC',
 'RJF',
 'RF',
 'STT',
 'SYF',
 'TFC',
 'USB',
 'WFC',
 'ZION']

It looks like the yfinance package was unable to find ebitda for these 28 companies as well. Given that it is missing from two different sources, I will proceed by just dropping this feature from the analysis.

In [ ]:
df = df.drop('EBITDA',axis=1)

In [ ]:
#same process for currentDebt
missing_debt =  df.loc[df['currentDebt'].isna(),'Symbol'].to_list()
debt = {}
#tickers = yf.Tickers(missing_ebitda)
#ebitda_df = pd.DataFrame(columns = ['webiste','ebitda','Symbol'])
for ticker in missing_debt:
  t = yf.Ticker(ticker)
  x = t.info
  y = pd.DataFrame(x)
  if 'totalDebt' not in y:
    continue
  else:
    #print(y.pegRatio)
    z = y.totalDebt.mean()
    debt[ticker] = z


debt = pd.DataFrame.from_dict(debt,orient='index', columns=['currentDebt']).reset_index()
debt.columns = ['Symbol','currentDebt']
df = df.merge(debt[['Symbol','currentDebt']],how='left', left_on = 'Symbol', right_on = 'Symbol')
df['currentDebt'] = df['currentDebt_x'].fillna(df['currentDebt_y'])
df = df.drop(['currentDebt_x','currentDebt_y'],axis=1)

In [ ]:
missing_debt =  df.loc[df['currentDebt'].isna(),'Symbol'].to_list()
print(missing_debt)

['INTU', 'MPWR']


In [ ]:
#manually fill in currentDebt for INTU MPWR
df.loc[df['Symbol'] == 'INTU', 'currentDebt'] = 6430000000
df.loc[df['Symbol'] == 'MPWR', 'currentDebt'] = 5780000




In [ ]:
#capital Expenditures is not in yfinance data so this column will now be dropped along with dividendPayout
df = df.drop(['capitalExpenditures','dividendPayout'],axis=1)

In [ ]:
#now I am left with features where 1-3 values are missing. I will now go manually fill these out using Google and yfinance for the specific company, and i will drop any columns not available in yfiancne
df.loc[df['Symbol'] == 'VLTO', 'BookValue'] = 17.89
df['BookValue']=df.BookValue.astype(float)
#Beta
df.loc[df['Symbol'] == 'VLTO', 'Beta'] = 1.43
df.loc[df['Symbol'] == 'KVUE', 'Beta'] = 0.64
df.loc[df['Symbol'] == 'GEHC', 'Beta'] = 1.03
df['Beta']=df.Beta.astype(float)
#Gross Profit
df = df.drop(['grossProfit'],axis=1)
#total revenue
df.loc[df['Symbol'] == 'VFC', 'totalRevenue'] = 6140000000
df['totalRevenue']=df.totalRevenue.astype(float)

df = df.drop(['costOfRevenue'],axis=1)

#operatingExpenses
df.loc[df['Symbol'] == 'VFC', 'operatingExpenses'] = 10680000000
df['operatingExpenses']=df.operatingExpenses.astype(float)

df = df.drop(['ebit','operatingCashflow','profitLoss'],axis=1)





In [ ]:
df.to_csv('final_data.csv')

In [ ]:
df.isna().sum()

Symbol                        0
MarketCapitalization          0
BookValue                     0
EPS                           0
GrossProfitTTM                0
DilutedEPSTTM                 0
QuarterlyEarningsGrowthYOY    0
QuarterlyRevenueGrowthYOY     0
TrailingPE                    0
Beta                          0
fiscalDateEnding              0
totalRevenue                  0
operatingIncome               0
operatingExpenses             0
netIncome                     0
totalAssets                   0
totalLiabilities              0
totalShareholderEquity        0
DividendPerShare              0
DividendYield                 0
PEGRatio                      0
currentDebt                   0
dtype: int64

In [ ]:
msft = yf.Ticker("VFC")

# get all stock info
msft.info

{'address1': '1551 Wewatta Street',
 'city': 'Denver',
 'state': 'CO',
 'zip': '80202',
 'country': 'United States',
 'phone': '720 778 4000',
 'website': 'https://www.vfc.com',
 'industry': 'Apparel Manufacturing',
 'industryKey': 'apparel-manufacturing',
 'industryDisp': 'Apparel Manufacturing',
 'sector': 'Consumer Cyclical',
 'sectorKey': 'consumer-cyclical',
 'sectorDisp': 'Consumer Cyclical',
 'longBusinessSummary': 'V.F. Corporation, together with its subsidiaries, engages in the design, procurement, marketing, and distribution of branded lifestyle apparel, footwear, and related products for men, women, and children in the Americas, Europe, and the Asia-Pacific. It operates through three segments: Outdoor, Active, and Work. The company offers outdoor, merino wool and other natural fibers-based, lifestyle, and casual apparel; footwear; equipment; accessories; outdoor-inspired, performance-based, youth culture/action sports-inspired, streetwear, and protective work footwear; handb